In [ ]:
import pandas as pd
import os
import re
from bs4 import BeautifulSoup
pd.set_option('display.max_rows', 50)

df = pd.DataFrame()

# category = "groupbuys"
# category = "hardware"
# category = "miners"
category = "mining"
# category = "mining_speculation"
# category = "mining_support"
# category = "pools"

# folder2 = "sorted-preprocessed-data"
folder2 = "raw-data"

#load every csv in the folder and append them
csv = os.listdir(folder2+"/"+category)
for file in csv:
    df = pd.concat([df, pd.read_csv(folder2+"/"+category+"/"+file)], axis=0)

df["original_info"] = df["original_info"].astype(str)
df = df.reset_index(drop=True)

#shorten df to test code
# df = df[:300]

def clean_row(row):
    html = row["original_info"]
    soup = BeautifulSoup(html, 'html.parser')
    #the date is in <div class="smalltext">
    date = soup.find("div", {"class": "smalltext"}).text
    #if the date contains "Last edit: ", use that instead
    if "Last edit:" in date:
        date = date.split("Last edit: ")[1]
    if " by " in date:
        date = date.split(" by ")[0]
    #delete <div class="quote">
    for quote in soup.find_all("div", {"class": "quote"}):
        quote.decompose()
    for quote in soup.find_all("div", {"class": "quoteheader"}):
        quote.decompose()
    #post is in <div class="post">
    post = soup.find("div", {"class": "post"}).text
    #remove all links
    post = re.sub(r'http\S+', '<link>', post)
    #remove duplicate spaces
    post = re.sub(' +', ' ', post)
    #remove all non-ascii characters
    post = post.encode("ascii", errors="ignore").decode()
    #remove all words that have a length above 20 (most likely a btc address or something useless)
    post = " ".join([(word if len(word) < 20 else "<truncated>") for word in post.split()])

    return post, date


In [ ]:
df["post"], df["date"] = zip(*df.apply(clean_row, axis=1))

In [ ]:
df = df[['topic', 'post', 'date']]
# replace "Today at " with "October 1, 2023, "
df["date"] = df["date"].str.replace("Today at ", "September 21, 2023, ")
df['date'] = pd.to_datetime(df['date'])
df = df.sort_values(by=["date"])
#remove "Re: " from topic
df["topic"] = df["topic"].str.replace("Re: ", "")
df

In [ ]:
# group by topic and append all the strings from the post column
df["date"] = df["date"].astype(str)
df = df.groupby('topic').agg({'post': '<sep>'.join, 'date': '<sep>'.join}).reset_index()

In [ ]:
df["dates"] = df["date"]
df['date'] = df['date'].apply(lambda x: x.split("<sep>")[-1])
df['date'] = pd.to_datetime(df['date'])
df = df.sort_values(by=["date"])

In [ ]:
df

In [ ]:
# import matplotlib.pyplot as plt
# plt.hist(df["start_edit"], bins=10, edgecolor='black')

# # Add labels and title
# plt.xlabel('date')
# plt.ylabel('Value')
# plt.title('Bar Plot of Integers')

# # Show the plot
# plt.show()

# print(df["start_edit"].max())
# print(df["start_edit"].min())

In [ ]:
import gzip
import pickle
#save to gzip
with gzip.open('cleaned-data/'+category+'.pkl.gz', 'wb') as f:
    pickle.dump(df, f)